In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "random_walk"
    cd(joinpath(@__DIR__, "../../../"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using MorphoMolNotebooks
using GLMakie
using LinearAlgebra
using Random
using Rotations
using StaticArrays
using Distributions
using Distances

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolNotebooks`
Precompiling MorphoMol
  ✓ MorphoMol
  1 dependency successfully precompiled in 3 seconds. 258 already precompiled.
[ Info: Precompiling MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83]
Precompiling MorphoMolNotebooks
  ✓ MorphoMolNotebooks
  1 dependency successfully precompiled in 53 seconds. 283 already precompiled.
[ Info: Precompiling MorphoMolNotebooks [3c5b58fc-d541-4bcd-a332-95372d8cdad8]
┌ Warning: Module MorphoMol with build ID fafbfcfd-5b03-0507-0000-1622c55db775 is missing from the cache.
│ This may mean MorphoMol [85e20460-a9b2-48f6-9df6-e154e9748d83] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing MorphoMolNotebooks [3c5b58fc-d541-4bcd-a332-95372d8cdad8].
[ Info: Precompiling JLD2 [033835bb-8acc-5ee8-8aae-3f567f8a3819]
[ Info: Precompiling Graphs [86223c79-3864-5bf0-83f7-82e725a168b6]
[ 

In [3]:
mol_type = "6r7m"
template_centers = MorphoMol.TEMPLATES[mol_type]["template_centers"]
template_radii = MorphoMol.TEMPLATES[mol_type]["template_radii"]
n_mol = 2
bounds = 80.0
x_init = MorphoMol.get_initial_state(n_mol, bounds)
n_atoms_per_mol = length(template_radii)
radii = vcat([template_radii for i in 1:n_mol]...);

In [5]:
T = 0.65
β = 1.0 / T

#persistence_weights = [0.5, -0.25, -0.1]
persistence_weights = [0.0, 1.0, 0.0]

σ_r = 0.15
σ_t = 1.25
Σ = vcat([[σ_r, σ_r, σ_r, σ_t, σ_t, σ_t] for _ in 1:n_mol]...)

input = Dict(
    "template_centers" => template_centers,
    "template_radii" => template_radii,
    "n_mol" => n_mol,
    "σ_r" => σ_r,
    "σ_t" => σ_t,
    "persistence_weights" => persistence_weights,
    "T" => T,
    "mol_type" => mol_type
)

energy(x) = MorphoMol.interface_persistence(x, template_centers, persistence_weights)
perturbation(x) = MorphoMol.perturb_single_randomly_chosen(x, σ_r, σ_t)

output = Dict{String, Vector}(
    "states" => Vector{Vector{Float64}}([]),
    "Es" => Vector{Float64}([]),
    "αs" => Vector{Float32}([]),
    "P0s" => Vector{Float64}([]),
    "P1s" => Vector{Float64}([]),
    "P2s" => Vector{Float64}([]),
)

rwm = MorphoMol.Algorithms.RandomWalkMetropolis(energy, perturbation, β)
simulation_time_minutes = 0.3

0.3

In [7]:
MorphoMol.Algorithms.simulate!(rwm, x_init, simulation_time_minutes, output);
output["αs"][end]

0.17391305f0

In [8]:
length(output["states"])

5

In [9]:
n = length(output["Es"])
e_mindex = argmin(output["Es"])

figure_config = Dict(
    "plot_ms" => 5,
    "tracker_ms" => 10,
    "title_fs" => 20,
    "vis_range" =>  maximum([1, e_mindex-750]):minimum([e_mindex + 50, n])
)

f = Figure(fontsize = 12)
#Slider
sl_i = Slider(f[3, 1:3], range = 1:length(output["states"][figure_config["vis_range"]]), startvalue = 1)
x = sl_i.value
cgl = GridLayout(f[1,1])
MorphoMolNotebooks.visualize_configuration_sequence!(cgl, x, input, output, figure_config)

# aspgl = GridLayout(f[2, 1:2])    
# MorphoMolNotebooks.visualize_alpha_shape_persistence_measures!(aspgl, x, input, output, figure_config)

etgl = GridLayout(f[2, 3])
MorphoMolNotebooks.visualize_energy_and_theta!(etgl, x, input, output, figure_config)

display(f)

GLMakie.Screen(...)

2024-12-20 17:33:51.140 julia[9247:287354] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-20 17:33:51.140 julia[9247:287354] +[IMKInputSession subclass]: chose IMKInputSession_Modern
